**Imports necesarios para el funcionamiento del codigo**


In [88]:
from __future__ import print_function
from collections import deque
from typing import Any, List, Optional
import pandas as pd
import numpy as np
!pip install graphviz
!apt-get install graphviz
import graphviz
import folium
from geopy.geocoders import Nominatim
from folium import plugins
from IPython.display import IFrame
from IPython.display import display, IFrame
from IPython.display import display, HTML
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


Metodos y clases:

In [90]:
import pandas as pd
import graphviz
m = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
class Nodo:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def __init__(self):
        self.root = None

    def get_height(self, node):
        if node is None:
            return 0
        return node.height

    def rotate_right(self, y):
        x = y.left
        T2 = x.right

        x.right = y
        y.left = T2

        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))

        return x

    def rotate_left(self, x):
        y = x.right
        T2 = y.left

        y.left = x
        x.right = T2

        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def double_rotate_right(self, node):
        node.left = self.rotate_left(node.left)
        return self.rotate_right(node)

    def double_rotate_left(self, node):
        node.right = self.rotate_right(node.right)
        return self.rotate_left(node)

    def get_balance_factor(self, node):
        if node is None:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)

    def insert(self, root, data):
        if root is None:
            return Nodo(data)

        if data['price'] / data['surface_total'] < root.data['price'] / root.data['surface_total']:
            root.left = self.insert(root.left, data)
        elif data['price'] / data['surface_total'] > root.data['price'] / root.data['surface_total']:
            root.right = self.insert(root.right, data)
        else:
            if data['longitude'] < root.data['longitude']:
                root.left = self.insert(root.left, data)
            else:
                root.right = self.insert(root.right, data)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance_factor(root)

        if balance > 1:
            if data['price'] / data['surface_total'] < root.left.data['price'] / root.left.data['surface_total']:
                return self.rotate_right(root)
            elif data['price'] / data['surface_total'] > root.left.data['price'] / root.left.data['surface_total']:
                root.left = self.rotate_left(root.left)
                return self.rotate_right(root)
        if balance < -1:
            if data['price'] / data['surface_total'] > root.right.data['price'] / root.right.data['surface_total']:
                return self.rotate_left(root)
            elif data['price'] / data['surface_total'] < root.right.data['price'] / root.right.data['surface_total']:
                root.right = self.rotate_right(root.right)
                return self.rotate_left(root)

        return root

    def insert_data(self, data):
        self.root = self.insert(self.root, data)

    def insert_node(self, title, department, city, property_type, latitude, longitude, surface_total, surface_covered, bedrooms, bathrooms, operation_type, price):
        node_data = {
            'title': title,
            'department': department,
            'city': city,
            'property_type': property_type,
            'latitude': latitude,
            'longitude': longitude,
            'surface_total': surface_total,
            'surface_covered': surface_covered,
            'bedrooms': bedrooms,
            'bathrooms': bathrooms,
            'operation_type': operation_type,
            'price': price
        }
        self.root = self.insert(self.root, node_data)

    def delete_node(self, target_metric, target_longitude, tolerance=0.1):
        self.root = self._delete_node(self.root, target_metric, target_longitude, tolerance)

    def _delete_node(self, root, target_metric, target_longitude, tolerance=0.1):
        if not root:
            return root

        if (
            abs(target_metric - root.data['price'] / root.data['surface_total']) <= tolerance
            and abs(target_longitude - root.data['longitude']) <= tolerance
        ):
            if not root.left:
                return root.right
            elif not root.right:
                return root.left
            temp = self._get_min_value_node(root.right)
            root.data = temp.data
            root.right = self._delete_node(root.right, target_metric, target_longitude, tolerance)

        if (target_metric - tolerance) < root.data['price'] / root.data['surface_total']:
            root.left = self._delete_node(root.left, target_metric, target_longitude, tolerance)
        else:
            root.right = self._delete_node(root.right, target_metric, target_longitude, tolerance)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance_factor(root)

        if balance > 1:
            if self.get_balance_factor(root.left) >= 0:
                return self.rotate_right(root)
            else:
                root.left = self.rotate_left(root.left)
                return self.rotate_right(root)
        if balance < -1:
            if self.get_balance_factor(root.right) <= 0:
                return self.rotate_left(root)
            else:
                root.right = self.rotate_right(root.right)
                return self.rotate_left(root)

        return root

    def search_by_metric(self, root, target_metric, target_longitude, tolerance=0.1):
        if root is None:
            return None

        current_metric = root.data['price'] / root.data['surface_total']
        current_longitude = root.data['longitude']

        if abs(target_metric - current_metric) <= tolerance:
            if abs(target_longitude - current_longitude) <= tolerance:
                return root
            elif target_longitude < current_longitude:
                return self.search_by_metric(root.left, target_metric, target_longitude, tolerance)
            else:
                return self.search_by_metric(root.right, target_metric, target_longitude, tolerance)
        elif target_metric < current_metric:
            return self.search_by_metric(root.left, target_metric, target_longitude, tolerance)
        else:
            return self.search_by_metric(root.right, target_metric, target_longitude, tolerance)

    # Punto 3: Función de búsqueda por métrica dada
    def search_node_by_metric(self, target_metric, target_longitude, tolerance=0.1):
         found_node3 = self.search_by_metric(self.root, target_metric, target_longitude, tolerance)
         return found_node3

     # Punto 4: Función para buscar todos los nodos que cumplan con una serie de criterios (máximo 3)
    def search_by_criteria(self, node, criteria):
        result = []

        if node is None:
            return result

        # Verificar los criterios y agregar nodos que cumplan con todos los criterios
        if all(node.data[key] == value for key, value in criteria.items()):
            result.append(node)

        # Realizar la búsqueda en los subárboles
        result += self.search_by_criteria(node.left, criteria)
        result += self.search_by_criteria(node.right, criteria)

        return result

    def search_nodes_by_criteria(self, criteria):
        return self.search_by_criteria(self.root, criteria)

    def print_balance_factors(self):
     if self.root is None:
        return

     stack = [(self.root, False)]

     while stack:
        node, visited = stack.pop()

        if visited:
            balance = self.get_balance_factor(node)
            print(f"Node: {node.data['title']} - Balance Factor: {balance}")
        else:
            if node.right:
                stack.append((node.right, False))
            stack.append((node, True))
            if node.left:
                stack.append((node.left, False))

    #Punto 5: Mostrar el recorrido por niveles de manera recursiva
    def print_level_order_recursive(self):
        if not self.root:
            return

        max_level = self.get_height(self.root)
        for level in range(1, max_level + 1):
            print(f"Nivel {level}:")
            self.print_nodes_at_level(self.root, level)

    def print_nodes_at_level(self, node, level):
        if node is None:
            return
        if level == 1:
            print(f"Title: {node.data['title']}, Price/Surface Total: {node.data['price'] / node.data['surface_total']:.2f}, Longitude: {node.data['longitude']}, Balance Factor: {self.get_balance_factor(node)}")
        elif level > 1:
            self.print_nodes_at_level(node.left, level - 1)
            self.print_nodes_at_level(node.right, level - 1)


    def _get_min_value_node(self, node):
        if not node or not node.left:
            return node
        return self._get_min_value_node(node.left)

    def to_dot(self):
        if not self.root:
            return None

        dot = graphviz.Digraph(format='png', engine='dot')
        self._to_dot(dot, self.root)
        return dot

    def _to_dot(self, dot, node):
        if node:
            label = f"Title: {node.data['title']}\nPrice/Surface Total: {node.data['price'] / node.data['surface_total']:.2f}\nLongitude: {node.data['longitude']}\nBalance Factor: {self.get_balance_factor(node)}"
            dot.node(str(id(node)), label)
            if node.left:
                dot.edge(str(id(node)), str(id(node.left)))
                self._to_dot(dot, node.left)
            if node.right:
                dot.edge(str(id(node)), str(id(node.right)))
                self._to_dot(dot, node.right)

    def print_tree_inserccion(self):
        dot = self.to_dot()
        if dot:
            dot.format = 'pdf'  # Establece el formato de salida a PDF
            dot.render('avl_tree_inserccion', view=True)
        else:
            print("El árbol está vacío.")

    def print_tree_eliminacion(self):
        dot = self.to_dot()
        if dot:
            dot.format = 'pdf'  # Establece el formato de salida a PDF
            dot.render('avl_tree_eliminacion', view=True)
        else:
            print("El árbol está vacío.")

    def print_tree(self):
        dot = self.to_dot()
        if dot:
            dot.format = 'pdf'
            dot.render('avl_tree', view=True)
        else:
            print("El árbol está vacío.")


            #Ojito con esto, son a b c d e

    def find_node_level(self, node, target_data, level=1):
        if node is None:
            return 0
        if node.data == target_data:
            return level
        left_level = self.find_node_level(node.left, target_data, level + 1)
        if left_level != 0:
            return left_level
        return self.find_node_level(node.right, target_data, level + 1)

    def get_node_level(self, target_data):
        if self.root is None:
            return None
        level = self.find_node_level(self.root, target_data)
        if level != 0:
            return level
        else:
            return None

#el papi
    def find_parent_node(self, node, target_data, parent=None):
        if node is None:
            return None
        if node.data == target_data:
            return parent
        left_parent = self.find_parent_node(node.left, target_data, node)
        if left_parent is not None:
            return left_parent
        return self.find_parent_node(node.right, target_data, node)

    def get_parent_node(self, target_data):
        if self.root is None:
            return None
        parent_node = self.find_parent_node(self.root, target_data)
        return parent_node

#ay abuelitooo (esto es encontrar el papa del papa)

    def find_grandparent_node(self, node, target_data):
        if node is None:
            return None
        parent_node = self.find_parent_node(node, target_data)
        if parent_node is not None:
            grandparent_node = self.find_parent_node(self.root, parent_node.data)
            return grandparent_node
        else:
            return None

    def get_grandparent_node(self, target_data):
        if self.root is None:
            return None
        grandparent_node = self.find_grandparent_node(self.root, target_data)
        return grandparent_node



    def find_uncle_node(self, node, target_data):
        if node is None:
            return None
        grandparent_node = self.find_grandparent_node(node, target_data)
        if grandparent_node is not None:
            parent_node = self.find_parent_node(node, target_data)
            if parent_node.data == grandparent_node.left.data:
                uncle_node = grandparent_node.right
            else:
                uncle_node = grandparent_node.left
            return uncle_node
        else:
            return None

    def get_uncle_node(self, target_data):
        if self.root is None:
            return None
        uncle_node = self.find_uncle_node(self.root, target_data)
        return uncle_node

    def generate_folium_map(latitude, longitude, address):
      m = folium.Map(location=[latitude, longitude], zoom_start=12)
      folium.Marker([latitude, longitude], tooltip=address).add_to(m)
      return m























**Creacion Del arbol AVL**


In [91]:
# Crear el árbol AVL
avl_tree = AVLTree()

# Cargar los datos desde el archivo CSV (reemplazar con tu ruta de archivo)
data = pd.read_csv("/content/co_properties_final.csv", sep=",",error_bad_lines=False)

# Insertar los datos en el Árbol AVL
for _, row in data.iterrows():
    property_info = {
        'title': row['title'],
        'department': row['department'],
        'city': row['city'],
        'property_type': row['property_type'],
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'surface_total': row['surface_total'],
        'surface_covered': row['surface_covered'],
        'bedrooms': row['bedrooms'],
        'bathrooms': row['bathrooms'],
        'operation_type': row['operation_type'],
        'price': row['price']
    }

    avl_tree.insert_data(property_info)
    avl_tree.print_tree()





<ipython-input-91-5ee501ab0bce>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/co_properties_final.csv", sep=",",error_bad_lines=False)
Skipping line 41: expected 12 fields, saw 13
Skipping line 126: expected 12 fields, saw 15



Menu iterativo

In [92]:
while True:
    print("\nMenú:")
    print("1. Insertar un nodo")
    print("2. Eliminar un nodo por métrica")
    print("3. Buscar un nodo por métrica")
    print("4. Buscar nodos por criterios")
    print("5. Mostrar recorrido por niveles")
    print("6. Operaciones con nodos obtenidos de 3 y 4")
    print("7. Salir")

    opcion = input("Selecciona una opción (1/2/3/4/5/6/7): ")

    if opcion == "1":

        print("Ingresa los datos del nodo a insertar:")
        # Solicitar los datos al usuario y realizar validaciones
        title = input("Título: ")
        department = input("Departamento: ")
        city = input("Ciudad: ")
        property_type = input("Tipo de propiedad: ")
        try:
            latitude = float(input("Latitud: "))
            longitude = float(input("Longitud: "))
            surface_total = float(input("Superficie total: "))
            surface_covered = float(input("Superficie cubierta: "))
            bedrooms = int(input("Número de habitaciones: "))
            bathrooms = int(input("Número de baños: "))
            operation_type = input("Tipo de operación: ")
            price = float(input("Precio: "))

            # Crear un diccionario con los datos ingresados
            node_data = {
                'title': title,
                'department': department,
                'city': city,
                'property_type': property_type,
                'latitude': latitude,
                'longitude': longitude,
                'surface_total': surface_total,
                'surface_covered': surface_covered,
                'bedrooms': bedrooms,
                'bathrooms': bathrooms,
                'operation_type': operation_type,
                'price': price
            }

            # Insertar el nodo en el árbol AVL
            avl_tree.insert_data(node_data)
            print("Nodo insertado correctamente.")

        except ValueError:
            print("Error: Los datos numéricos deben ser números válidos.")

        avl_tree.print_tree_inserccion()


        pass

    elif opcion == "2":
      # Pedir al usuario las métricas para eliminar un nodo
      target_metric = float(input("Ingrese la métrica (Price/Surface Total) del nodo a eliminar: "))
      target_longitude = float(input("Ingrese la longitud del nodo a eliminar: "))

      # Eliminar el nodo con las métricas ingresadas por el usuario
      avl_tree.delete_node(target_metric, target_longitude)
      print("Nodo eliminado")

      # Imprimir el árbol actualizado
      avl_tree.print_tree_eliminacion()

      pass
    elif opcion == "3":
        try:
            metric = float(input("Ingrese la métrica (price/surface_total): "))
            longitude = float(input("Ingrese la longitud: "))
            result3 = avl_tree.search_node_by_metric(metric, longitude)

            if result3:
                if isinstance(result3, list):

                    print("Nodos encontrados que cumplen con ambos criterios:")
                    for node in result3:
                        print(f"Title: {node.data['title']}\nCity: {node.data['city']}\nLongitude: {node.data['longitude']}\n")











                else:
                    print("Nodo encontrado que cumple con ambos criterios:")
                    print(f"Title: {result3.data['title']}\nCity: {result3.data['city']}\nLongitude: {result3.data['longitude']}\n")
                    nodo_seleccionado = result3.data

                    # Crea una instancia de geolocalización









            else:
                print("No se encontraron nodos que cumplan con los criterios especificados.")
        except ValueError:
            print("Por favor, ingrese valores numéricos válidos.")
        pass

    elif opcion == "4":

        # Menú de selección de criterios
        print("Seleccione hasta tres criterios para buscar propiedades:")
        print("1. Título")
        print("2. Departamento")
        print("3. Ciudad")
        print("4. Tipo de propiedad")
        print("5. Latitud")
        print("6. Longitud")
        print("7. Superficie Total")
        print("8. Superficie Cubierta")
        print("9. Dormitorios")
        print("10. Baños")
        print("11. Tipo de Operación")
        print("12. Precio")

        selected_criteria = []
        while len(selected_criteria) < 3:
            choice = input("Seleccione un criterio (ingrese el número correspondiente) o presione Enter para continuar: ")
            if not choice:
                break
            if choice.isdigit():
                choice = int(choice)
                if 1 <= choice <= 12:
                    selected_criteria.append(choice)
                else:
                    print("Selección no válida. Ingrese un número del 1 al 12.")

        # Solicitar valores para los criterios seleccionados
        criteria = {}
        for choice in selected_criteria:
            if choice == 1:
                criteria['title'] = input("Ingrese el título: ")
            elif choice == 2:
                criteria['department'] = input("Ingrese el departamento: ")
            elif choice == 3:
                criteria['city'] = input("Ingrese la ciudad: ")
            elif choice == 4:
                criteria['property_type'] = input("Ingrese el tipo de propiedad: ")
            elif choice == 5:
                criteria['latitude'] = float(input("Ingrese la latitud: "))
            elif choice == 6:
                criteria['longitude'] = float(input("Ingrese la longitud: "))
            elif choice == 7:
                criteria['surface_total'] = float(input("Ingrese la superficie total: "))
            elif choice == 8:
                criteria['surface_covered'] = float(input("Ingrese la superficie cubierta: "))
            elif choice == 9:
                criteria['bedrooms'] = int(input("Ingrese la cantidad de dormitorios: "))
            elif choice == 10:
                criteria['bathrooms'] = int(input("Ingrese la cantidad de baños: "))
            elif choice == 11:
                criteria['operation_type'] = input("Ingrese el tipo de operación: ")
            elif choice == 12:
                criteria['price'] = float(input("Ingrese el precio: "))

        # Realizar la búsqueda por criterios
        matching_nodes = avl_tree.search_nodes_by_criteria(criteria)

        # Imprimir los nodos que cumplen con los criterios
        if matching_nodes:
            print("Nodos encontrados que cumplen con los criterios:")
            for node in matching_nodes:
                print(f"Title: {node.data['title']}\nCity: {node.data['city']}\nBedrooms: {node.data['bedrooms']}\nPrice: {node.data['price']}\nsurface_total:{node.data['surface_total']}\n")
        else:
            print("No se encontraron nodos que cumplan con los criterios especificados.")
        pass

    elif opcion == "5":
        print("Recorrido por niveles:")
        avl_tree.print_level_order_recursive()
        pass

    elif opcion == "6":
        selected_node = None

        while True:
            print("\nOperaciones adicionales:")
            print("a. Seleccionar un nodo")
            print("b. Realizar operaciones en el nodo seleccionado")
            print("c. Regresar al menú principal")

            opcion_adicional = input("Selecciona una opción (a/b/c): ")

            if opcion_adicional == "a":
                while True:
                    print("\nSelecciona la fuente del nodo:")
                    print("1. Utilizar nodo encontrado previamente (found_node3)")
                    print("2. Seleccionar un nodo de la lista (result[])")

                    opcion_fuente = input("Selecciona una fuente (1/2): ")

                    if opcion_fuente == "1":
                        # Aquí selecciona el nodo result3
                        selected_node = result3
                        print(f"Nodo seleccionado: {selected_node}")
                        geolocalizador = Nominatim(user_agent="miGeocodificador")

                    # Geolocaliza el nodo seleccionado utilizando la latitud y la longitud
                        latitud = nodo_seleccionado['latitude']
                        longitud = nodo_seleccionado['longitude']
                        try:
                          ubicacion = geolocalizador.reverse(f"{latitud}, {longitud}", language="es")
                          print("Información de la ubicación:")
                          print(ubicacion.address)

                     # Genera el mapa de Folium y lo muestra en la celda de Colab
                          folium_map = folium.Map(location=[latitud, longitud], zoom_start=12)
                          folium.Marker([latitud, longitud], tooltip=ubicacion.address).add_to(folium_map)
                          plugins.Fullscreen(position='topright', force_separate_button=True).add_to(folium_map)
                          display(folium_map)
                        except GeocoderTimedOut:
                          print("El servicio de geocodificación se agotó. Por favor, inténtalo de nuevo.")



                        break
                    elif opcion_fuente == "2":
                      # Mostrar la lista de nodos disponibles en result[]
                          print("Nodos disponibles en result[]:")
                          for i, nodo in enumerate(matching_nodes, start=1):
                              print(f"{i}. {nodo.data}")



                          # Solicitar al usuario que seleccione un nodo por número
                          while True:
                              try:
                                  seleccion = int(input("Selecciona un nodo por número o presiona 0 para volver al menú principal: "))

                                  if seleccion == 0:
                                      break  # Salir del bucle y volver al menú principal
                                  elif 1 <= seleccion <= len(matching_nodes):
                                      selected_node = matching_nodes[seleccion - 1]
                                      print(f"Nodo seleccionado: {selected_node.data}")
                                      geolocalizador = Nominatim(user_agent="miGeocodificador")
                                      latitud = selected_node.data['latitude']
                                      longitud = selected_node.data['longitude']
                                      try:
                                        ubicacion = geolocalizador.reverse(f"{latitud}, {longitud}", language="es")
                                        print("Información de la ubicación:")
                                        print(ubicacion.address)

                                    # Genera el mapa de Folium y lo muestra en la celda de Colab
                                        folium_map = folium.Map(location=[latitud, longitud], zoom_start=12)
                                        folium.Marker([latitud, longitud], tooltip=ubicacion.address).add_to(folium_map)
                                        plugins.Fullscreen(position='topright', force_separate_button=True).add_to(folium_map)
                                        display(folium_map)
                                      except GeocoderTimedOut:
                                        print("El servicio de geocodificación se agotó. Por favor, inténtalo de nuevo.")

                                      break
                                  else:
                                      print("Número fuera de rango. Por favor, elige un número válido.")
                              except ValueError:
                                  print("Entrada no válida. Por favor, ingresa un número válido.")
                          break


            elif opcion_adicional == "b":
                # Opción para realizar operaciones en un nodo seleccionado
                if selected_node is None:
                    print("Ningún nodo ha sido seleccionado. Por favor, selecciona un nodo primero.")
                else:
                    while True:
                        print("\nOperaciones en el nodo seleccionado:")
                        print("1. Obtener el nivel del nodo")
                        print("2. Obtener el factor de balanceo del nodo")
                        print("3. Encontrar el padre del nodo")
                        print("4. Encontrar el abuelo del nodo")
                        print("5. Encontrar el tío del nodo")
                        print("6. Regresar al menú anterior")

                        opcion_operacion = input("Selecciona una operación (1/2/3/4/5/6): ")

                        if opcion_operacion == "1":

                            if selected_node:
                                    node_level = avl_tree.get_node_level(selected_node.data)
                                    if node_level is not None:
                                        print(f"Nivel del nodo seleccionado: {node_level}")

                                    else:
                                        print("El nodo seleccionado no se encuentra en el árbol.")
                            else:
                                    print("Ningún nodo ha sido seleccionado. Por favor, selecciona un nodo primero.")
                            # Llama a una función para obtener el nivel del nodo utilizando selected_node
                            pass
                        elif opcion_operacion == "2":
                            if selected_node:
                                    balance_factorb = -1*avl_tree.get_balance_factor(selected_node)
                                    if balance_factorb is not None:
                                        print(f"Factor de Balanceo del nodo seleccionado: {balance_factorb}")
                                    else:
                                        print("El nodo seleccionado no se encuentra en el árbol.")
                            else:
                                    print("Ningún nodo ha sido seleccionado. Por favor, selecciona un nodo primero.")


                            # Llama a una función para obtener el factor de balanceo del nodo utilizando selected_node
                            pass
                        elif opcion_operacion == "3":
                            # Llama a una función para encontrar el padre del nodo utilizando selected_node
                            if selected_node:
                                    parent_node = avl_tree.get_parent_node(selected_node.data)
                                    if parent_node is not None:
                                        print(f"El padre del nodo seleccionado es: {parent_node.data}")
                                        geolocalizador = Nominatim(user_agent="miGeocodificador")
                                        latitud = parent_node.data['latitude']
                                        longitud = parent_node.data['longitude']
                                        try:
                                          ubicacion = geolocalizador.reverse(f"{latitud}, {longitud}", language="es")
                                          print("Información de la ubicación:")
                                          print(ubicacion.address)

                                    # Genera el mapa de Folium y lo muestra en la celda de Colab
                                          folium_map = folium.Map(location=[latitud, longitud], zoom_start=12)
                                          folium.Marker([latitud, longitud], tooltip=ubicacion.address).add_to(folium_map)
                                          plugins.Fullscreen(position='topright', force_separate_button=True).add_to(folium_map)
                                          display(folium_map)
                                        except GeocoderTimedOut:
                                          print("El servicio de geocodificación se agotó. Por favor, inténtalo de nuevo.")

                                    else:
                                        print("El nodo seleccionado no tiene un padre (es la raíz del árbol o no se encuentra en el árbol).")
                            else:
                                    print("Ningún nodo ha sido seleccionado. Por favor, selecciona un nodo primero.")


                            pass
                        elif opcion_operacion == "4":
                            # Llama a una función para encontrar el abuelo del nodo utilizando selected_node
                            if selected_node:
                                    grandparent_node = avl_tree.get_grandparent_node(selected_node.data)
                                    if grandparent_node is not None:
                                        print(f"El abuelo del nodo seleccionado es: {grandparent_node.data}")
                                        geolocalizador = Nominatim(user_agent="miGeocodificador")
                                        latitud = grandparent_node.data['latitude']
                                        longitud = grandparent_node.data['longitude']
                                        try:
                                          ubicacion = geolocalizador.reverse(f"{latitud}, {longitud}", language="es")
                                          print("Información de la ubicación:")
                                          print(ubicacion.address)

                                    # Genera el mapa de Folium y lo muestra en la celda de Colab
                                          folium_map = folium.Map(location=[latitud, longitud], zoom_start=12)
                                          folium.Marker([latitud, longitud], tooltip=ubicacion.address).add_to(folium_map)
                                          plugins.Fullscreen(position='topright', force_separate_button=True).add_to(folium_map)
                                          display(folium_map)
                                        except GeocoderTimedOut:
                                          print("El servicio de geocodificación se agotó. Por favor, inténtalo de nuevo.")
                                    else:
                                        print("El nodo seleccionado no tiene un abuelo (es la raíz del árbol, su padre no tiene padre o no se encuentra en el árbol).")
                            else:
                                    print("Ningún nodo ha sido seleccionado. Por favor, selecciona un nodo primero.")


                            pass
                        elif opcion_operacion == "5":
                            if selected_node:
                                    uncle_node = avl_tree.get_uncle_node(selected_node.data)
                                    if uncle_node is not None:
                                        print(f"El tío del nodo seleccionado es: {uncle_node.data}")
                                        geolocalizador = Nominatim(user_agent="miGeocodificador")
                                        latitud = uncle_node.data['latitude']
                                        longitud = uncle_node.data['longitude']
                                        try:
                                          ubicacion = geolocalizador.reverse(f"{latitud}, {longitud}", language="es")
                                          print("Información de la ubicación:")
                                          print(ubicacion.address)

                                    # Genera el mapa de Folium y lo muestra en la celda de Colab
                                          folium_map = folium.Map(location=[latitud, longitud], zoom_start=12)
                                          folium.Marker([latitud, longitud], tooltip=ubicacion.address).add_to(folium_map)
                                          plugins.Fullscreen(position='topright', force_separate_button=True).add_to(folium_map)
                                          display(folium_map)
                                        except GeocoderTimedOut:
                                          print("El servicio de geocodificación se agotó. Por favor, inténtalo de nuevo.")
                                    else:
                                        print("El nodo seleccionado no tiene un tío (no tiene abuelo, su padre no tiene hermano o no se encuentra en el árbol).")
                            else:
                                    print("Ningún nodo ha sido seleccionado. Por favor, selecciona un nodo primero.")

                            pass
                        elif opcion_operacion == "6":
                            # Regresar al menú anterior
                            break
                        else:
                            print("Opción no válida. Por favor, selecciona una opción válida.")

            elif opcion_adicional == "c":
                # Regresar al menú principal
                break

            else:
                print("Opción no válida. Por favor, selecciona una opción válida.")


    elif opcion == "7":
        # Opción para salir del programa
        break

    else:
        print("Opción no válida. Por favor, selecciona una opción válida.")



Menú:
1. Insertar un nodo
2. Eliminar un nodo por métrica
3. Buscar un nodo por métrica
4. Buscar nodos por criterios
5. Mostrar recorrido por niveles
6. Operaciones con nodos obtenidos de 3 y 4
7. Salir
Selecciona una opción (1/2/3/4/5/6/7): 3
Ingrese la métrica (price/surface_total): 18853.70
Ingrese la longitud: -74.815
Nodo encontrado que cumple con ambos criterios:
Title: Casa En Arriendo/venta En Barranquilla Caujaral Cod. AREI-16605
City: Barranquilla
Longitude: -74.815


Menú:
1. Insertar un nodo
2. Eliminar un nodo por métrica
3. Buscar un nodo por métrica
4. Buscar nodos por criterios
5. Mostrar recorrido por niveles
6. Operaciones con nodos obtenidos de 3 y 4
7. Salir
Selecciona una opción (1/2/3/4/5/6/7): 6

Operaciones adicionales:
a. Seleccionar un nodo
b. Realizar operaciones en el nodo seleccionado
c. Regresar al menú principal
Selecciona una opción (a/b/c): a

Selecciona la fuente del nodo:
1. Utilizar nodo encontrado previamente (found_node3)
2. Seleccionar un nodo d


Operaciones adicionales:
a. Seleccionar un nodo
b. Realizar operaciones en el nodo seleccionado
c. Regresar al menú principal
Selecciona una opción (a/b/c): c

Menú:
1. Insertar un nodo
2. Eliminar un nodo por métrica
3. Buscar un nodo por métrica
4. Buscar nodos por criterios
5. Mostrar recorrido por niveles
6. Operaciones con nodos obtenidos de 3 y 4
7. Salir
Selecciona una opción (1/2/3/4/5/6/7): 7
